In [2]:
import pandas as pd
import io, requests, os 
from bs4 import BeautifulSoup
from tqdm import tqdm
pd.set_option('display.max_columns', None)

output_folder = 'CovidMay17-2022'
if not os.path.exists(output_folder):
    os.makedirs(output_folder, exist_ok=True)

def valid_date(date):
    try:
        pd.to_datetime(date)
        return True
    except:
        return False

# PVI data

In [2]:
# read in fips codes
fips = pd.read_csv(
    "https://github.com/COVID19PVI/data/raw/master/Model11.2.1/data/Model_11.2.1_20200228_data.csv", 
    header=12, usecols=["name","casrn"]
).rename(columns={'name':'Name','casrn':'FIPS'})

# get the PVI data links
r = requests.get("https://github.com/COVID19PVI/data/tree/master/Model11.2.1")
soup = BeautifulSoup(r.content, 'html.parser')
links = [
    x.attrs["href"].replace("blob","raw")
    for x in soup.find_all('a', {'class':'js-navigation-open Link--primary'})[1:-1]
]

In [3]:
# combine data
all_data = []
read_from_local = True # Set to true if github isn't responding

if read_from_local:
    filenames = [link.split('/')[-1] for link in links]
    model_path = '../../data/Model11.2.1/' # PVI git repo cloned here in local machine
    for filename in tqdm(filenames):
        new_data = pd.read_csv(os.path.join(model_path, filename))
        new_data["date"] = filename.split("_")[2]
        all_data.append(new_data)

else:
    for link in tqdm(links):
        new_data = pd.read_csv("https://github.com"+link)
        new_data["date"] = link.split("_")[2]
        all_data.append(new_data)

all_data = pd.concat(all_data)
all_data["date"] = [f"{i[:4]}-{i[4:6]}-{i[6:]}" for i in all_data.date]
all_data.head()

100%|██████████| 973/973 [00:12<00:00, 75.43it/s]


,ToxPi Score,HClust Group,KMeans Group,Name,Source,Infection Rate: Transmissible Cases!25!0xcc3333ff,Infection Rate: Disease Spread!5!0xe64d4dff,Pop Concentration: Pop Mobility!10!0x57b757ff,Pop Concentration: Residential Density!10!0x5ced5cff,Intervention: Social Distancing!10!0x4258c9ff,Intervention: Testing!10!0x6079f7ff,Health & Environment: Pop Demographics!10!0x6b0b9eff,Health & Environment: Air Pollution!10!0x8e26c4ff,Health & Environment: Age Distribution!10!0x9a42c8ff,Health & Environment: Co-morbidities!10!0xb460e0ff,Health & Environment: Health Disparities!10!0xc885ecff,Health & Environment: Hospital Beds!5!0xdeb9f1ff,date
0,0.659739,1,1,"California, Solano","-121.9357,38.2683",1.000000,1.000000,0.717074,0.6905,1.000,0.504366,0.622993,0.479042,0.504195,0.236237,0.235189,0.514283,2020-02-28
1,0.624339,2,1,"California, Madera","-119.7666,37.2151",0.638652,1.000000,0.601609,0.8239,0.825,0.504366,0.605671,0.658683,0.469686,0.321223,0.590624,0.613680,2020-02-28
2,0.598916,1,1,"California, Santa Cruz","-122.0067,37.0558",0.817887,1.000000,0.704589,0.9035,1.000,0.504366,0.409785,0.269461,0.511724,0.100021,0.272557,0.531471,2020-02-28
3,0.590967,2,1,"California, Imperial","-115.3669,33.0393",0.591782,0.666667,0.585525,0.9869,0.750,0.504366,0.525758,0.628743,0.443774,0.270404,0.620412,0.516830,2020-02-28
4,0.578017,1,1,"California, Santa Clara","-121.697,37.231",0.830152,0.880000,0.793513,0.7310,1.000,0.504366,0.452019,0.353293,0.452613,0.060895,0.131427,0.461418,2020-02-28


In [4]:
drop_columns = [
    'Name', 'Source','date', 'ToxPi Score', 'HClust Group', 'KMeans Group', 'Hospital Beds',
    'Pop Mobility', 'Co-morbidities', 'Pop Demographics', 'Air Pollution', 'Residential Density', 'Testing',
    'Age Distribution', 'Health Disparities', 'Transmissible Cases'
]
for column in all_data.columns:
    column_name = column.split('!')[0]
    if ":" in column_name:
        column_name = column_name.split(": ")[1]

    if column_name in drop_columns:
        continue
    print(f"Saving {column_name}", end = "... ")
    mx = all_data.pivot(index="Name",columns="date", values=column).reset_index()
    mx = pd.merge(mx, fips, on="Name", how="left")
    mx.insert(0, "FIPS", mx.pop("FIPS"))
    mx.to_csv(f"{output_folder}/{column_name}.csv", index=False, float_format='%.4f')
    print(u'\u2705')

Saving Transmissible Cases... ✅
Saving Disease Spread... ✅
Saving Pop Mobility... ✅
Saving Residential Density... ✅
Saving Social Distancing... ✅
Saving Testing... ✅
Saving Pop Demographics... ✅
Saving Air Pollution... ✅
Saving Age Distribution... ✅
Saving Co-morbidities... ✅
Saving Health Disparities... ✅


In [5]:
all_data.columns

Index(['ToxPi Score', 'HClust Group', 'KMeans Group', 'Name', 'Source',
       'Infection Rate: Transmissible Cases!25!0xcc3333ff',
       'Infection Rate: Disease Spread!5!0xe64d4dff',
       'Pop Concentration: Pop Mobility!10!0x57b757ff',
       'Pop Concentration: Residential Density!10!0x5ced5cff',
       'Intervention: Social Distancing!10!0x4258c9ff',
       'Intervention: Testing!10!0x6079f7ff',
       'Health & Environment: Pop Demographics!10!0x6b0b9eff',
       'Health & Environment: Air Pollution!10!0x8e26c4ff',
       'Health & Environment: Age Distribution!10!0x9a42c8ff',
       'Health & Environment: Co-morbidities!10!0xb460e0ff',
       'Health & Environment: Health Disparities!10!0xc885ecff',
       'Health & Environment: Hospital Beds!5!0xdeb9f1ff', 'date'],
      dtype='object')

# Cases
Source [USAFacts](https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/). Data collection starts from 2020-01-22. Note that data from today can take 3 days to update in this file.

In [2]:
response = requests.get("https://static.usafacts.org/public/data/covid-19/covid_confirmed_usafacts.csv")

file_object = io.StringIO(response.content.decode('utf-8'))
df = pd.read_csv(file_object)
df.head()

,countyFIPS,County Name,State,StateFIPS,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,...,2022-10-16,2022-10-17,2022-10-18,2022-10-19,2022-10-20,2022-10-21,2022-10-22,2022-10-23,2022-10-24,2022-10-25
0,0,Statewide Unallocated,AL,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1001,Autauga County,AL,1,0,0,0,0,0,0,...,18452,18452,18452,18452,18480,18480,18480,18480,18480,18480
2,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,65819,65819,65819,65819,65895,65895,65895,65895,65895,65895
3,1005,Barbour County,AL,1,0,0,0,0,0,0,...,6910,6910,6910,6910,6926,6926,6926,6926,6926,6926
4,1007,Bibb County,AL,1,0,0,0,0,0,0,...,7547,7547,7547,7547,7560,7560,7560,7560,7560,7560


In [3]:
df.drop(columns=['County Name', 'State', 'StateFIPS'], inplace=True)
df.rename({'countyFIPS':'FIPS'}, axis=1, inplace=True)

df = df[(~df['FIPS'].isna()) & (df['FIPS']>0)]
df.fillna(0, inplace=True)
df.to_csv(f'{output_folder}/Cases.csv', index=False)

# Transmissible Cases

In [3]:
cases = pd.read_csv(f'{output_folder}/Cases.csv')
population = pd.read_csv('Support files/Population.csv')

In [4]:
df = population[['FIPS', 'POPESTIMATE']].merge(cases, how='inner', on='FIPS')
date_columns = [col for col in df.columns if valid_date(col)]
print(df.shape)
df.head(3)

(3142, 857)


,FIPS,POPESTIMATE,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,2020-01-31,2020-02-01,2020-02-02,2020-02-03,2020-02-04,2020-02-05,2020-02-06,2020-02-07,2020-02-08,2020-02-09,2020-02-10,2020-02-11,2020-02-12,2020-02-13,2020-02-14,2020-02-15,2020-02-16,2020-02-17,2020-02-18,2020-02-19,2020-02-20,2020-02-21,2020-02-22,2020-02-23,2020-02-24,2020-02-25,2020-02-26,2020-02-27,2020-02-28,2020-02-29,2020-03-01,2020-03-02,2020-03-03,2020-03-04,2020-03-05,2020-03-06,2020-03-07,2020-03-08,2020-03-09,2020-03-10,2020-03-11,2020-03-12,2020-03-13,2020-03-14,2020-03-15,2020-03-16,2020-03-17,2020-03-18,2020-03-19,2020-03-20,2020-03-21,2020-03-22,2020-03-23,2020-03-24,2020-03-25,2020-03-26,2020-03-27,2020-03-28,2020-03-29,2020-03-30,2020-03-31,2020-04-01,2020-04-02,2020-04-03,2020-04-04,2020-04-05,2020-04-06,2020-04-07,2020-04-08,2020-04-09,2020-04-10,2020-04-11,2020-04-12,2020-04-13,2020-04-14,2020-04-15,2020-04-16,2020-04-17,2020-04-18,2020-04-19,2020-04-20,2020-04-21,2020-04-22,2020-04-23,2020-04-24,2020-04-25,2020-04-26,2020-04-27,2020-04-28,2020-04-29,2020-04-30,2020-05-01,2020-05-02,2020-05-03,2020-05-04,2020-05-05,2020-05-06,2020-05-07,2020-05-08,2020-05-09,2020-05-10,2020-05-11,2020-05-12,2020-05-13,2020-05-14,2020-05-15,2020-05-16,2020-05-17,2020-05-18,2020-05-19,2020-05-20,2020-05-21,2020-05-22,2020-05-23,2020-05-24,2020-05-25,2020-05-26,2020-05-27,2020-05-28,2020-05-29,2020-05-30,2020-05-31,2020-06-01,2020-06-02,2020-06-03,2020-06-04,2020-06-05,2020-06-06,2020-06-07,2020-06-08,2020-06-09,2020-06-10,2020-06-11,2020-06-12,2020-06-13,2020-06-14,2020-06-15,2020-06-16,2020-06-17,2020-06-18,2020-06-19,2020-06-20,2020-06-21,2020-06-22,2020-06-23,2020-06-24,2020-06-25,2020-06-26,2020-06-27,2020-06-28,2020-06-29,2020-06-30,2020-07-01,2020-07-02,2020-07-03,2020-07-04,2020-07-05,2020-07-06,2020-07-07,2020-07-08,2020-07-09,2020-07-10,2020-07-11,2020-07-12,2020-07-13,2020-07-14,2020-07-15,2020-07-16,2020-07-17,2020-07-18,2020-07-19,2020-07-20,2020-07-21,2020-07-22,2020-07-23,2020-07-24,2020-07-25,2020-07-26,2020-07-27,2020-07-28,2020-07-29,2020-07-30,2020-07-31,2020-08-01,2020-08-02,2020-08-03,2020-08-04,2020-08-05,2020-08-06,2020-08-07,2020-08-08,2020-08-09,2020-08-10,2020-08-11,2020-08-12,2020-08-13,2020-08-14,2020-08-15,2020-08-16,2020-08-17,2020-08-18,2020-08-19,2020-08-20,2020-08-21,2020-08-22,2020-08-23,2020-08-24,2020-08-25,2020-08-26,2020-08-27,2020-08-28,2020-08-29,2020-08-30,2020-08-31,2020-09-01,2020-09-02,2020-09-03,2020-09-04,2020-09-05,2020-09-06,2020-09-07,2020-09-08,2020-09-09,2020-09-10,2020-09-11,2020-09-12,2020-09-13,2020-09-14,2020-09-15,2020-09-16,2020-09-17,2020-09-18,2020-09-19,2020-09-20,2020-09-21,2020-09-22,2020-09-23,2020-09-24,2020-09-25,2020-09-26,2020-09-27,2020-09-28,2020-09-29,2020-09-30,2020-10-01,2020-10-02,2020-10-03,2020-10-04,2020-10-05,2020-10-06,2020-10-07,2020-10-08,2020-10-09,2020-10-10,2020-10-11,2020-10-12,2020-10-13,2020-10-14,2020-10-15,2020-10-16,2020-10-17,2020-10-18,2020-10-19,2020-10-20,2020-10-21,2020-10-22,2020-10-23,2020-10-24,2020-10-25,2020-10-26,2020-10-27,2020-10-28,2020-10-29,2020-10-30,2020-10-31,2020-11-01,2020-11-02,2020-11-03,2020-11-04,2020-11-05,2020-11-06,2020-11-07,2020-11-08,2020-11-09,2020-11-10,2020-11-11,2020-11-12,2020-11-13,2020-11-14,2020-11-15,2020-11-16,2020-11-17,2020-11-18,2020-11-19,2020-11-20,2020-11-21,2020-11-22,2020-11-23,2020-11-24,2020-11-25,2020-11-26,2020-11-27,2020-11-28,2020-11-29,2020-11-30,2020-12-01,2020-12-02,2020-12-03,2020-12-04,2020-12-05,2020-12-06,2020-12-07,2020-12-08,2020-12-09,2020-12-10,2020-12-11,2020-12-12,2020-12-13,2020-12-14,2020-12-15,2020-12-16,2020-12-17,2020-12-18,2020-12-19,2020-12-20,2020-12-21,2020-12-22,2020-12-23,2020-12-24,2020-12-25,2020-12-26,2020-12-27,2020-12-28,2020-12-29,2020-12-30,2020-12-31,2021-01-01,2021-01-02,2021-01-03,2021-01-04,2021-01-05,2021-01-06,2021-01-07,2021-01-08,2021-01-09,2021-01-10,2021-01-11,2021-01-12,2021-01-13,2021-01-14,2021-01-15,2021-01-16,2021-01-17,

In [5]:
period = 14
forward_shifted = df[date_columns].shift(period, axis=1).fillna(0)

cases_in_last_14_days = df[date_columns[period:]] - forward_shifted[date_columns[period:]]
df[date_columns[period:]] = cases_in_last_14_days.divide(df['POPESTIMATE']/1e5, axis=0)
df[date_columns[:period]] = df[date_columns[:period]].divide(df['POPESTIMATE']/1e5, axis=0
)
df.drop(columns=['POPESTIMATE'], axis=1, inplace=True)
df.head(3)

,FIPS,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,2020-01-31,2020-02-01,2020-02-02,2020-02-03,2020-02-04,2020-02-05,2020-02-06,2020-02-07,2020-02-08,2020-02-09,2020-02-10,2020-02-11,2020-02-12,2020-02-13,2020-02-14,2020-02-15,2020-02-16,2020-02-17,2020-02-18,2020-02-19,2020-02-20,2020-02-21,2020-02-22,2020-02-23,2020-02-24,2020-02-25,2020-02-26,2020-02-27,2020-02-28,2020-02-29,2020-03-01,2020-03-02,2020-03-03,2020-03-04,2020-03-05,2020-03-06,2020-03-07,2020-03-08,2020-03-09,2020-03-10,2020-03-11,2020-03-12,2020-03-13,2020-03-14,2020-03-15,2020-03-16,2020-03-17,2020-03-18,2020-03-19,2020-03-20,2020-03-21,2020-03-22,2020-03-23,2020-03-24,2020-03-25,2020-03-26,2020-03-27,2020-03-28,2020-03-29,2020-03-30,2020-03-31,2020-04-01,2020-04-02,2020-04-03,2020-04-04,2020-04-05,2020-04-06,2020-04-07,2020-04-08,2020-04-09,2020-04-10,2020-04-11,2020-04-12,2020-04-13,2020-04-14,2020-04-15,2020-04-16,2020-04-17,2020-04-18,2020-04-19,2020-04-20,2020-04-21,2020-04-22,2020-04-23,2020-04-24,2020-04-25,2020-04-26,2020-04-27,2020-04-28,2020-04-29,2020-04-30,2020-05-01,2020-05-02,2020-05-03,2020-05-04,2020-05-05,2020-05-06,2020-05-07,2020-05-08,2020-05-09,2020-05-10,2020-05-11,2020-05-12,2020-05-13,2020-05-14,2020-05-15,2020-05-16,2020-05-17,2020-05-18,2020-05-19,2020-05-20,2020-05-21,2020-05-22,2020-05-23,2020-05-24,2020-05-25,2020-05-26,2020-05-27,2020-05-28,2020-05-29,2020-05-30,2020-05-31,2020-06-01,2020-06-02,2020-06-03,2020-06-04,2020-06-05,2020-06-06,2020-06-07,2020-06-08,2020-06-09,2020-06-10,2020-06-11,2020-06-12,2020-06-13,2020-06-14,2020-06-15,2020-06-16,2020-06-17,2020-06-18,2020-06-19,2020-06-20,2020-06-21,2020-06-22,2020-06-23,2020-06-24,2020-06-25,2020-06-26,2020-06-27,2020-06-28,2020-06-29,2020-06-30,2020-07-01,2020-07-02,2020-07-03,2020-07-04,2020-07-05,2020-07-06,2020-07-07,2020-07-08,2020-07-09,2020-07-10,2020-07-11,2020-07-12,2020-07-13,2020-07-14,2020-07-15,2020-07-16,2020-07-17,2020-07-18,2020-07-19,2020-07-20,2020-07-21,2020-07-22,2020-07-23,2020-07-24,2020-07-25,2020-07-26,2020-07-27,2020-07-28,2020-07-29,2020-07-30,2020-07-31,2020-08-01,2020-08-02,2020-08-03,2020-08-04,2020-08-05,2020-08-06,2020-08-07,2020-08-08,2020-08-09,2020-08-10,2020-08-11,2020-08-12,2020-08-13,2020-08-14,2020-08-15,2020-08-16,2020-08-17,2020-08-18,2020-08-19,2020-08-20,2020-08-21,2020-08-22,2020-08-23,2020-08-24,2020-08-25,2020-08-26,2020-08-27,2020-08-28,2020-08-29,2020-08-30,2020-08-31,2020-09-01,2020-09-02,2020-09-03,2020-09-04,2020-09-05,2020-09-06,2020-09-07,2020-09-08,2020-09-09,2020-09-10,2020-09-11,2020-09-12,2020-09-13,2020-09-14,2020-09-15,2020-09-16,2020-09-17,2020-09-18,2020-09-19,2020-09-20,2020-09-21,2020-09-22,2020-09-23,2020-09-24,2020-09-25,2020-09-26,2020-09-27,2020-09-28,2020-09-29,2020-09-30,2020-10-01,2020-10-02,2020-10-03,2020-10-04,2020-10-05,2020-10-06,2020-10-07,2020-10-08,2020-10-09,2020-10-10,2020-10-11,2020-10-12,2020-10-13,2020-10-14,2020-10-15,2020-10-16,2020-10-17,2020-10-18,2020-10-19,2020-10-20,2020-10-21,2020-10-22,2020-10-23,2020-10-24,2020-10-25,2020-10-26,2020-10-27,2020-10-28,2020-10-29,2020-10-30,2020-10-31,2020-11-01,2020-11-02,2020-11-03,2020-11-04,2020-11-05,2020-11-06,2020-11-07,2020-11-08,2020-11-09,2020-11-10,2020-11-11,2020-11-12,2020-11-13,2020-11-14,2020-11-15,2020-11-16,2020-11-17,2020-11-18,2020-11-19,2020-11-20,2020-11-21,2020-11-22,2020-11-23,2020-11-24,2020-11-25,2020-11-26,2020-11-27,2020-11-28,2020-11-29,2020-11-30,2020-12-01,2020-12-02,2020-12-03,2020-12-04,2020-12-05,2020-12-06,2020-12-07,2020-12-08,2020-12-09,2020-12-10,2020-12-11,2020-12-12,2020-12-13,2020-12-14,2020-12-15,2020-12-16,2020-12-17,2020-12-18,2020-12-19,2020-12-20,2020-12-21,2020-12-22,2020-12-23,2020-12-24,2020-12-25,2020-12-26,2020-12-27,2020-12-28,2020-12-29,2020-12-30,2020-12-31,2021-01-01,2021-01-02,2021-01-03,2021-01-04,2021-01-05,2021-01-06,2021-01-07,2021-01-08,2021-01-09,2021-01-10,2021-01-11,2021-01-12,2021-01-13,2021-01-14,2021-01-15,2021-01-16,2021-01-17,2021-01-18,2

In [6]:
df.round(3).to_csv(f'{output_folder}/Transmissible Cases.csv', index=False)

# Disease Spread

In [7]:
df = pd.read_csv(f'{output_folder}/Cases.csv')

In [8]:
period = 14
shifted = df[date_columns].shift(period, axis=1).fillna(0)
cases_difference = df[date_columns[period:]] - shifted[date_columns[period:]]
df_14 = df.copy()
df_14[date_columns[period:]] = cases_difference

period = 28
shifted = df[date_columns].shift(period, axis=1).fillna(0)
cases_difference = df[date_columns[period:]] - shifted[date_columns[period:]]
df_28 = df.copy()
df_28[date_columns[period:]] = cases_difference

In [9]:
df[date_columns] = df_14[date_columns].divide(df_28[date_columns], axis=1).fillna(0)
df.head(3)

,FIPS,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,2020-01-31,2020-02-01,2020-02-02,2020-02-03,2020-02-04,2020-02-05,2020-02-06,2020-02-07,2020-02-08,2020-02-09,2020-02-10,2020-02-11,2020-02-12,2020-02-13,2020-02-14,2020-02-15,2020-02-16,2020-02-17,2020-02-18,2020-02-19,2020-02-20,2020-02-21,2020-02-22,2020-02-23,2020-02-24,2020-02-25,2020-02-26,2020-02-27,2020-02-28,2020-02-29,2020-03-01,2020-03-02,2020-03-03,2020-03-04,2020-03-05,2020-03-06,2020-03-07,2020-03-08,2020-03-09,2020-03-10,2020-03-11,2020-03-12,2020-03-13,2020-03-14,2020-03-15,2020-03-16,2020-03-17,2020-03-18,2020-03-19,2020-03-20,2020-03-21,2020-03-22,2020-03-23,2020-03-24,2020-03-25,2020-03-26,2020-03-27,2020-03-28,2020-03-29,2020-03-30,2020-03-31,2020-04-01,2020-04-02,2020-04-03,2020-04-04,2020-04-05,2020-04-06,2020-04-07,2020-04-08,2020-04-09,2020-04-10,2020-04-11,2020-04-12,2020-04-13,2020-04-14,2020-04-15,2020-04-16,2020-04-17,2020-04-18,2020-04-19,2020-04-20,2020-04-21,2020-04-22,2020-04-23,2020-04-24,2020-04-25,2020-04-26,2020-04-27,2020-04-28,2020-04-29,2020-04-30,2020-05-01,2020-05-02,2020-05-03,2020-05-04,2020-05-05,2020-05-06,2020-05-07,2020-05-08,2020-05-09,2020-05-10,2020-05-11,2020-05-12,2020-05-13,2020-05-14,2020-05-15,2020-05-16,2020-05-17,2020-05-18,2020-05-19,2020-05-20,2020-05-21,2020-05-22,2020-05-23,2020-05-24,2020-05-25,2020-05-26,2020-05-27,2020-05-28,2020-05-29,2020-05-30,2020-05-31,2020-06-01,2020-06-02,2020-06-03,2020-06-04,2020-06-05,2020-06-06,2020-06-07,2020-06-08,2020-06-09,2020-06-10,2020-06-11,2020-06-12,2020-06-13,2020-06-14,2020-06-15,2020-06-16,2020-06-17,2020-06-18,2020-06-19,2020-06-20,2020-06-21,2020-06-22,2020-06-23,2020-06-24,2020-06-25,2020-06-26,2020-06-27,2020-06-28,2020-06-29,2020-06-30,2020-07-01,2020-07-02,2020-07-03,2020-07-04,2020-07-05,2020-07-06,2020-07-07,2020-07-08,2020-07-09,2020-07-10,2020-07-11,2020-07-12,2020-07-13,2020-07-14,2020-07-15,2020-07-16,2020-07-17,2020-07-18,2020-07-19,2020-07-20,2020-07-21,2020-07-22,2020-07-23,2020-07-24,2020-07-25,2020-07-26,2020-07-27,2020-07-28,2020-07-29,2020-07-30,2020-07-31,2020-08-01,2020-08-02,2020-08-03,2020-08-04,2020-08-05,2020-08-06,2020-08-07,2020-08-08,2020-08-09,2020-08-10,2020-08-11,2020-08-12,2020-08-13,2020-08-14,2020-08-15,2020-08-16,2020-08-17,2020-08-18,2020-08-19,2020-08-20,2020-08-21,2020-08-22,2020-08-23,2020-08-24,2020-08-25,2020-08-26,2020-08-27,2020-08-28,2020-08-29,2020-08-30,2020-08-31,2020-09-01,2020-09-02,2020-09-03,2020-09-04,2020-09-05,2020-09-06,2020-09-07,2020-09-08,2020-09-09,2020-09-10,2020-09-11,2020-09-12,2020-09-13,2020-09-14,2020-09-15,2020-09-16,2020-09-17,2020-09-18,2020-09-19,2020-09-20,2020-09-21,2020-09-22,2020-09-23,2020-09-24,2020-09-25,2020-09-26,2020-09-27,2020-09-28,2020-09-29,2020-09-30,2020-10-01,2020-10-02,2020-10-03,2020-10-04,2020-10-05,2020-10-06,2020-10-07,2020-10-08,2020-10-09,2020-10-10,2020-10-11,2020-10-12,2020-10-13,2020-10-14,2020-10-15,2020-10-16,2020-10-17,2020-10-18,2020-10-19,2020-10-20,2020-10-21,2020-10-22,2020-10-23,2020-10-24,2020-10-25,2020-10-26,2020-10-27,2020-10-28,2020-10-29,2020-10-30,2020-10-31,2020-11-01,2020-11-02,2020-11-03,2020-11-04,2020-11-05,2020-11-06,2020-11-07,2020-11-08,2020-11-09,2020-11-10,2020-11-11,2020-11-12,2020-11-13,2020-11-14,2020-11-15,2020-11-16,2020-11-17,2020-11-18,2020-11-19,2020-11-20,2020-11-21,2020-11-22,2020-11-23,2020-11-24,2020-11-25,2020-11-26,2020-11-27,2020-11-28,2020-11-29,2020-11-30,2020-12-01,2020-12-02,2020-12-03,2020-12-04,2020-12-05,2020-12-06,2020-12-07,2020-12-08,2020-12-09,2020-12-10,2020-12-11,2020-12-12,2020-12-13,2020-12-14,2020-12-15,2020-12-16,2020-12-17,2020-12-18,2020-12-19,2020-12-20,2020-12-21,2020-12-22,2020-12-23,2020-12-24,2020-12-25,2020-12-26,2020-12-27,2020-12-28,2020-12-29,2020-12-30,2020-12-31,2021-01-01,2021-01-02,2021-01-03,2021-01-04,2021-01-05,2021-01-06,2021-01-07,2021-01-08,2021-01-09,2021-01-10,2021-01-11,2021-01-12,2021-01-13,2021-01-14,2021-01-15,2021-01-16,2021-01-17,2021-01-18,2

In [10]:
df.round(4).to_csv(f'{output_folder}/Disease Spread.csv', index=False)

# Deaths
Source [USAFacts](https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/). Data collection starts from 2020-01-22.

In [4]:
# response = requests.get("https://static.usafacts.org/public/data/covid-19/covid_deaths_usafacts.csv")
# file_object = io.StringIO(response.content.decode('utf-8'))
# df = pd.read_csv(file_object)
# df.head()

,countyFIPS,County Name,State,StateFIPS,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,...,2022-10-16,2022-10-17,2022-10-18,2022-10-19,2022-10-20,2022-10-21,2022-10-22,2022-10-23,2022-10-24,2022-10-25
0,0,Statewide Unallocated,AL,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1001,Autauga County,AL,1,0,0,0,0,0,0,...,228,228,228,228,228,228,228,228,228,228
2,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,715,715,715,715,716,716,716,716,716,716
3,1005,Barbour County,AL,1,0,0,0,0,0,0,...,103,103,103,103,103,103,103,103,103,103
4,1007,Bibb County,AL,1,0,0,0,0,0,0,...,108,108,108,108,108,108,108,108,108,108


In [5]:
# df.drop(columns=['County Name', 'State', 'StateFIPS'], inplace=True)
# df.rename({'countyFIPS':'FIPS'}, axis=1, inplace=True)

# df = df[(~df['FIPS'].isna()) & (df['FIPS']>0)]
# df.fillna(0, inplace=True)
# df.to_csv(f'{output_folder}/Deaths.csv', index=False)

# Vaccination
From [CDC vaccinations in the US counties](https://data.cdc.gov/Vaccinations/COVID-19-Vaccinations-in-the-United-States-County/8xkx-amqh). Data collection starts from 2020-12-13. Note that data from today can take upto 3 days to update in the CDC server.

In [6]:
response = requests.get("https://data.cdc.gov/api/views/8xkx-amqh/rows.csv?accessType=DOWNLOAD")
file_object = io.StringIO(response.content.decode('utf-8'))
df = pd.read_csv(file_object, low_memory=False)
df.head()

,Date,FIPS,MMWR_week,Recip_County,Recip_State,Completeness_pct,Administered_Dose1_Recip,Administered_Dose1_Pop_Pct,Administered_Dose1_Recip_5Plus,Administered_Dose1_Recip_5PlusPop_Pct,...,Booster_Doses_Vax_Pct_UR_Equity,Booster_Doses_12PlusVax_Pct_UR_Equity,Booster_Doses_18PlusVax_Pct_UR_Equity,Booster_Doses_65PlusVax_Pct_UR_Equity,Census2019,Census2019_5PlusPop,Census2019_5to17Pop,Census2019_12PlusPop,Census2019_18PlusPop,Census2019_65PlusPop
0,10/26/2022,16031,43,Cassia County,ID,97.7,12334.0,51.3,12322.0,55.8,...,6.0,6.0,6.0,6.0,24030.0,22097.0,5654.0,19113.0,16443.0,3473.0
1,10/26/2022,48345,43,Motley County,TX,98.9,470.0,39.2,470.0,41.2,...,7.0,7.0,7.0,6.0,1200.0,1141.0,178.0,1061.0,963.0,368.0
2,10/26/2022,38055,43,McLean County,ND,92.7,5412.0,57.3,5404.0,61.0,...,8.0,8.0,8.0,7.0,9450.0,8866.0,1494.0,8048.0,7372.0,2291.0
3,10/26/2022,17151,43,Pope County,IL,98.2,1859.0,44.5,1855.0,45.6,...,8.0,8.0,8.0,7.0,4177.0,4066.0,389.0,3902.0,3677.0,1183.0
4,10/26/2022,21209,43,Scott County,KY,94.1,37146.0,65.2,36907.0,69.2,...,3.0,3.0,4.0,3.0,57004.0,53310.0,10592.0,47684.0,42718.0,7050.0


In [7]:
df = df[df['FIPS']!='UNK']
df['FIPS'] = df['FIPS'].astype(int)
df = df[['Date', 'FIPS', 'Series_Complete_Pop_Pct']]

df['Date'] = pd.to_datetime(df['Date'])
# df = df.sort_values(by=['FIPS', 'Date']).reset_index(drop=True)
df.fillna(0, inplace=True)

In [10]:
df = df[['Date', 'FIPS', 'Series_Complete_Pop_Pct']].pivot_table(
    values='Series_Complete_Pop_Pct',
    index=['FIPS'],
    columns='Date'
).reset_index().rename_axis(None, axis=1)
df.head()

,FIPS,2020-12-13 00:00:00,2020-12-14 00:00:00,2020-12-15 00:00:00,2020-12-16 00:00:00,2020-12-17 00:00:00,2020-12-18 00:00:00,2020-12-19 00:00:00,2020-12-20 00:00:00,2020-12-21 00:00:00,...,2022-08-24 00:00:00,2022-08-31 00:00:00,2022-09-07 00:00:00,2022-09-14 00:00:00,2022-09-21 00:00:00,2022-09-28 00:00:00,2022-10-05 00:00:00,2022-10-12 00:00:00,2022-10-19 00:00:00,2022-10-26 00:00:00
0,1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,45.0,45.0,45.1,45.1,45.1,45.2,45.3,45.3,45.4,45.5
1,1003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,52.1,52.2,52.2,52.2,52.3,52.3,52.4,52.5,52.5,52.6
2,1005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,46.6,46.7,46.7,46.7,46.7,46.7,46.8,46.8,46.9,46.9
3,1007,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,35.8,35.8,35.8,35.8,35.8,35.9,35.9,36.0,36.0,36.0
4,1009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,32.4,32.4,32.4,32.5,32.5,32.5,32.6,32.6,32.6,32.6


In [ ]:
def fill_missing_dates(data):
    df = data.copy()
    df = df.fillna(method='ffill').fillna(0)
    # this assumes the dates are along the column axis
    date_columns = sorted([pd.to_datetime(col) for col in df.columns if valid_date(col)])
    non_date_columns = [col for col in df.columns if col not in date_columns]
    df.columns = non_date_columns + date_columns

    total_days = (date_columns[1] - date_columns[0]).days
    missing_dates = []

    previous_date = date_columns[0]
    for date in date_columns[1:]:
        steps = (date - previous_date).days
        if steps > 1:
            for step in range(1, steps):
                missing_dates.append(previous_date+pd.to_timedelta(step, unit='D'))
        previous_date = date

    print(f'Number of missing dates {len(len(missing_dates))}')

    df = df.reindex(columns = ['FIPS'] + sorted(date_columns + missing_dates))
    df = df.T.fillna(method='ffill').T
    return df

In [28]:
# this was added because after May 2022, 
# vaccination data from CDC started having random missing dates
df = fill_missing_dates(df)
df.head()

,FIPS,2020-12-13 00:00:00,2020-12-14 00:00:00,2020-12-15 00:00:00,2020-12-16 00:00:00,2020-12-17 00:00:00,2020-12-18 00:00:00,2020-12-19 00:00:00,2020-12-20 00:00:00,2020-12-21 00:00:00,2020-12-22 00:00:00,2020-12-23 00:00:00,2020-12-24 00:00:00,2020-12-25 00:00:00,2020-12-26 00:00:00,2020-12-27 00:00:00,2020-12-28 00:00:00,2020-12-29 00:00:00,2020-12-30 00:00:00,2020-12-31 00:00:00,2021-01-01 00:00:00,2021-01-02 00:00:00,2021-01-03 00:00:00,2021-01-04 00:00:00,2021-01-05 00:00:00,2021-01-06 00:00:00,2021-01-07 00:00:00,2021-01-08 00:00:00,2021-01-09 00:00:00,2021-01-10 00:00:00,2021-01-11 00:00:00,2021-01-12 00:00:00,2021-01-13 00:00:00,2021-01-14 00:00:00,2021-01-15 00:00:00,2021-01-16 00:00:00,2021-01-17 00:00:00,2021-01-18 00:00:00,2021-01-19 00:00:00,2021-01-20 00:00:00,2021-01-21 00:00:00,2021-01-22 00:00:00,2021-01-23 00:00:00,2021-01-24 00:00:00,2021-01-25 00:00:00,2021-01-26 00:00:00,2021-01-27 00:00:00,2021-01-28 00:00:00,2021-01-29 00:00:00,2021-01-30 00:00:00,2021-01-31 00:00:00,2021-02-01 00:00:00,2021-02-02 00:00:00,2021-02-03 00:00:00,2021-02-04 00:00:00,2021-02-05 00:00:00,2021-02-06 00:00:00,2021-02-07 00:00:00,2021-02-08 00:00:00,2021-02-09 00:00:00,2021-02-10 00:00:00,2021-02-11 00:00:00,2021-02-12 00:00:00,2021-02-13 00:00:00,2021-02-14 00:00:00,2021-02-15 00:00:00,2021-02-16 00:00:00,2021-02-17 00:00:00,2021-02-18 00:00:00,2021-02-19 00:00:00,2021-02-20 00:00:00,2021-02-21 00:00:00,2021-02-22 00:00:00,2021-02-23 00:00:00,2021-02-24 00:00:00,2021-02-25 00:00:00,2021-02-26 00:00:00,2021-02-27 00:00:00,2021-02-28 00:00:00,2021-03-01 00:00:00,2021-03-02 00:00:00,2021-03-03 00:00:00,2021-03-04 00:00:00,2021-03-05 00:00:00,2021-03-06 00:00:00,2021-03-07 00:00:00,2021-03-08 00:00:00,2021-03-09 00:00:00,2021-03-10 00:00:00,2021-03-11 00:00:00,2021-03-12 00:00:00,2021-03-13 00:00:00,2021-03-14 00:00:00,2021-03-15 00:00:00,2021-03-16 00:00:00,2021-03-17 00:00:00,2021-03-18 00:00:00,2021-03-19 00:00:00,2021-03-20 00:00:00,2021-03-21 00:00:00,2021-03-22 00:00:00,2021-03-23 00:00:00,2021-03-24 00:00:00,2021-03-25 00:00:00,2021-03-26 00:00:00,2021-03-27 00:00:00,2021-03-28 00:00:00,2021-03-29 00:00:00,2021-03-30 00:00:00,2021-03-31 00:00:00,2021-04-01 00:00:00,2021-04-02 00:00:00,2021-04-03 00:00:00,2021-04-04 00:00:00,2021-04-05 00:00:00,2021-04-06 00:00:00,2021-04-07 00:00:00,2021-04-08 00:00:00,2021-04-09 00:00:00,2021-04-10 00:00:00,2021-04-11 00:00:00,2021-04-12 00:00:00,2021-04-13 00:00:00,2021-04-14 00:00:00,2021-04-15 00:00:00,2021-04-16 00:00:00,2021-04-17 00:00:00,2021-04-18 00:00:00,2021-04-19 00:00:00,2021-04-20 00:00:00,2021-04-21 00:00:00,2021-04-22 00:00:00,2021-04-23 00:00:00,2021-04-24 00:00:00,2021-04-25 00:00:00,2021-04-26 00:00:00,2021-04-27 00:00:00,2021-04-28 00:00:00,2021-04-29 00:00:00,2021-04-30 00:00:00,2021-05-01 00:00:00,2021-05-02 00:00:00,2021-05-03 00:00:00,2021-05-04 00:00:00,2021-05-05 00:00:00,2021-05-06 00:00:00,2021-05-07 00:00:00,2021-05-08 00:00:00,2021-05-09 00:00:00,2021-05-10 00:00:00,2021-05-11 00:00:00,2021-05-12 00:00:00,2021-05-13 00:00:00,2021-05-14 00:00:00,2021-05-15 00:00:00,2021-05-16 00:00:00,2021-05-17 00:00:00,2021-05-18 00:00:00,2021-05-19 00:00:00,2021-05-20 00:00:00,2021-05-21 00:00:00,2021-05-22 00:00:00,2021-05-23 00:00:00,2021-05-24 00:00:00,2021-05-25 00:00:00,2021-05-26 00:00:00,2021-05-27 00:00:00,2021-05-28 00:00:00,2021-05-29 00:00:00,2021-05-30 00:00:00,2021-05-31 00:00:00,2021-06-01 00:00:00,2021-06-02 00:00:00,2021-06-03 00:00:00,2021-06-04 00:00:00,2021-06-05 00:00:00,2021-06-06 00:00:00,2021-06-07 00:00:00,2021-06-08 00:00:00,2021-06-09 00:00:00,2021-06-10 00:00:00,2021-06-11 00:00:00,2021-06-12 00:00:00,2021-06-13 00:00:00,2021-06-14 00:00:00,2021-06-15 00:00:00,2021-06-16 00:00:00,2021-06-17 00:00:00,2021-06-18 00:00:00,2021-06-19 00:00:00,2021-06-20 00:00:00,2021-06-21 00:00:00,2021-06-22 00:00:00,2021-06-23 00:00:00,2021-06-24 00:00:00,2021-06-25 00:00:00,2021-06-26 00:00:00,2021-06-27 00:00:00,2021-06-28 00:00:00,2021-06-29 00:00:00,2021-06-30 00:

In [29]:
df.to_csv(f'{output_folder}/Vaccination.csv', index=False)